# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [31]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [32]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [33]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [34]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [35]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [36]:
# creata a keyspace
try:
    session.execute(
    """
    CREATE KEYSPACE sparkify WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 1};
    """
    )

except Exception as e:
    print(e)


Keyspace 'sparkify' already exists


#### Set Keyspace

In [37]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [38]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
    
query = "CREATE TABLE IF NOT EXISTS artists_by_sessionid_iteminsession "
query = query + "(artist_name text, song text,firstName text, lastName text, length text, sessionId int, itemInSession int, PRIMARY KEY (sessionId,  itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [39]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sparkify.artists_by_sessionid_iteminsession (artist_name , song ,firstName , lastName , length , sessionId , itemInSession )"
        query = query + " VALUES (%s, %s, %s,%s, %s, %s,%s);"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            #print(line)
            session.execute(query, (line[0], line[9],line[1], line[4],line[5], int(line[8]), int(line[3])))
        except Exception as e:
            print(e)
        

#### Do a SELECT to verify that the data have been inserted into each table

In [40]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT * FROM artists_by_sessionid_iteminsession WHERE sessionId = 338 and itemInSession = 4;"
#query = query + "(artist_name text, song text,firstName text, lastName text, length text, sessionId int, itemInSession int, PRIMARY KEY (sessionId,  itemInSession))"
try:
    artist_library = session.execute(query)
except Exception as e:
    print(e)

for row in enumerate(artist_library):
    print(row)

(0, Row(sessionid=338, iteminsession=4, artist_name='Faithless', firstname='Ava', lastname='Robinson', length='495.3073', song='Music Matters (Mark Knight Dub)'))


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [41]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS artists_by_userid_sessionid_sortby_iteminsession "
query = query + "(artist_name text, song text,firstName text, lastName text, length text, sessionId int, itemInSession int,userid int, PRIMARY KEY ((userid,sessionId),itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)
    

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sparkify.artists_by_userid_sessionid_sortby_iteminsession (artist_name , song ,firstName , lastName , length , sessionId , itemInSession, userid )"
        query = query + " VALUES (%s, %s, %s,%s, %s, %s, %s, %s);"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            #print(line)
            session.execute(query, (line[0], line[9],line[1], line[4],line[5], int(line[8]), int(line[3]),int(line[10])))
        except Exception as e:
            print(e)
                            

query = "SELECT artist_name,song,userid,firstName,lastName FROM artists_by_userid_sessionid_sortby_iteminsession where userid = 10 and sessionId = 182 ;"
#query = query + "(artist_name text, song text,firstName text, lastName text, length text, sessionId int, itemInSession int, PRIMARY KEY (sessionId,  itemInSession))"
try:
    artist_library = session.execute(query)
except Exception as e:
    print(e)

for row in enumerate(artist_library):
    print(row)
    



                    

(0, Row(artist_name='Down To The Bone', song="Keep On Keepin' On", userid=10, firstname='Sylvie', lastname='Cruz'))
(1, Row(artist_name='Three Drives', song='Greece 2000', userid=10, firstname='Sylvie', lastname='Cruz'))
(2, Row(artist_name='Sebastien Tellier', song='Kilometer', userid=10, firstname='Sylvie', lastname='Cruz'))
(3, Row(artist_name='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', userid=10, firstname='Sylvie', lastname='Cruz'))


In [42]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS artists_by_song_userid "
query = query + "(artist_name text, song text,firstName text, lastName text, length text, sessionId int, itemInSession int,userid int, PRIMARY KEY (song,userid));"
try:
    session.execute(query)
except Exception as e:
    print(e)
    

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sparkify.artists_by_song_userid (artist_name , song ,firstName , lastName , length , sessionId , itemInSession, userid )"
        query = query + " VALUES (%s, %s, %s,%s, %s, %s, %s, %s);"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            #print(line)
            session.execute(query, (line[0], line[9],line[1], line[4],line[5], int(line[8]), int(line[3]),int(line[10])))
        except Exception as e:
            print(e)
                            

query = "SELECT userid,firstName,lastName FROM artists_by_song_userid where song = 'All Hands Against His Own' ;"
#query = query + "(artist_name text, song text,firstName text, lastName text, length text, sessionId int, itemInSession int, PRIMARY KEY (sessionId,  itemInSession))"
try:
    artist_library = session.execute(query)
except Exception as e:
    print(e)

for row in enumerate(artist_library):
    print(row)
                    

(0, Row(userid=29, firstname='Jacqueline', lastname='Lynch'))
(1, Row(userid=80, firstname='Tegan', lastname='Levine'))
(2, Row(userid=95, firstname='Sara', lastname='Johnson'))


### Drop the tables before closing out the sessions

In [44]:
## Drop the table before closing out the sessions

query = "DROP TABLE artists_by_sessionid_iteminsession;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE artists_by_userid_sessionid_sortby_iteminsession;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE artists_by_song_userid;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

  

### Close the session and cluster connection¶

In [45]:
## Close the session and cluster connection
session.shutdown()
cluster.shutdown()